# Basic Residual Block (ResNet) — Baştan Sona Detaylı Notlar

Bu notların amacı: **basic residual block** mantığını sıfırdan kurup, hem sezgisel hem de uygulamaya dönük şekilde öğrenmek.  
Konu bittiğinde şu üç soruya net cevap verebiliyor olacaksın:

1) *Skip connection neden var, neyi çözüyor?*  
2) *Identity shortcut ile projection shortcut farkı ne?*  
3) *PyTorch’ta “doğru” residual block nasıl yazılır ve hangi hatalar sık yapılır?*

---

## 0) Residual fikri “neden” ortaya çıktı?

Derin ağları (çok katmanlı CNN’leri) büyüttükçe iki büyük problem artar:

- **Optimization zorluğu**: Ağ derinleşince eğitim zorlaşır, kayıp (loss) düşmesi yavaşlar veya takılır.  
- **Degradation (bozulma) problemi**: Daha derin bir ağ, teoride daha iyi olmalı gibi görünür; ama pratikte bazen **daha kötü** performans verir.

Bu ikinci sorun çok önemli: *Bu, overfitting değil.*  
Overfitting’de eğitim hatası düşer ama test bozulur. Degradation’da ise **eğitim hatası bile düşmeyebilir**.

Residual bağlantının ana fikri şudur:

> “Ağa yeni bir şey öğrenmeyi zorla; ama olmazsa en azından kimlik (identity) gibi davranıp performansı bozmamasını sağla.”

---

## 1) Residual öğrenme: `H(x)` yerine `F(x)`

Klasik bir blok şunu öğrenmeye çalışır:

- Girdi: `x`
- Çıkış hedefi: `H(x)`

Residual blok ise şunu yapar:

- Öğrenilecek şey: `F(x) = H(x) - x`
- Sonuç: `H(x) = F(x) + x`

Yani blok **doğrudan H(x)’i** öğrenmiyor; `x`’in üzerine eklenecek “düzeltmeyi” öğreniyor.

Bu sayede:
- Eğer “düzeltme” gerekmiyorsa, `F(x) ≈ 0` olur ve blok **x’i aynen geçirir**.
- Derin ağda her katmanın “sıfırdan bir fonksiyon” öğrenmesi yerine, küçük düzeltmeler öğrenmesi daha kolay olur.

---

## 2) Basic residual block’un matematiksel formu

En temel form: 

\begin{aligned}
y &= F(x) + x
\end{aligned}

- `x`: blok girişi. x, residual bloktan önceki feature map, yani **bloğun girdisi (input tensor)**dur
- `F(x)`: blok içindeki conv’lar + aktivasyonlar + normalizasyonun oluşturduğu dönüşüm
- `+ x`: skip connection (shortcut)

Bu toplama işlemi element-wise yapılır. Bu yüzden **boyutlar eşit olmalı**:
- Kanal sayısı (C)
- Yükseklik (H)
- Genişlik (W)

Boyut eşitliği bozulursa, `x` doğrudan eklenemez — işte burada projection shortcut devreye girer (aşağıda).

**Residual’da toplama, öğrenilmesi gereken fonksiyonu sadeleştirir ve gradyanın bozulmadan akmasını sağlar.**

---

## 3) Basic Residual Block yapısı (ResNet-18 / ResNet-34)

“Basic” denince genelde şu blok kastedilir:

- 3×3 Conv → BN → ReLU
- 3×3 Conv → BN
- Skip ekle (`+ x`)
- ReLU

Şematik:

```bash
x ────────────────┐
                  │
      3×3 Conv     │
      BN           │
      ReLU         │
      3×3 Conv     │
      BN           │
                  (+) → ReLU → y
                  │
x (shortcut) ──────┘
```

Burada dikkat:
- İlk conv genelde `stride=1` (aynı çözünürlük) veya blok geçişlerinde `stride=2` (downsample) olabilir.
- İkinci conv genelde `stride=1` kalır.
- Toplamadan sonra bir ReLU uygulanır (basic ResNet’in orijinal tasarımında).

---

## 4) Identity shortcut vs Projection shortcut

### 4.1 Identity shortcut (en basit ve en iyi senaryo)

Boyutlar aynıysa:

- `shortcut(x) = x`

Yani skip path üzerinde hiçbir katman yok.

Avantajları:
- Parametre eklemez
- Gradient akışı çok temiz olur

### 4.2 Projection shortcut (boyut eşleme gerektiğinde)

Boyutlar farklıysa (örneğin stride=2 ile H,W yarıya düşürülmüşse veya kanal sayısı değişmişse):

- `shortcut(x) = W_s * x` şeklinde bir dönüşüm gerekir.
- Pratikte çoğunlukla **1×1 convolution** kullanılır:

`Conv1x1(stride=s, out_channels=...)`

Bu durumda blok:

\begin{aligned}
y = F(x) + \text{proj}(x)
\end{aligned}

Bu 1×1 conv hem:
- Kanal sayısını eşler
- Gerekirse stride ile çözünürlüğü düşürür

---

## 5) Downsampling (stage geçişi) mantığı


**Downsampling, bir feature map’in uzamsal çözünürlüğünü (H, W) bilinçli olarak düşürme işlemidir.**

ResNet mimarisinde çözünürlük stage stage düşer. Örnek:

- Stage1: 56×56
- Stage2: 28×28
- Stage3: 14×14
- Stage4: 7×7

Stage geçişinde genelde ilk blok:
- Main path’te `stride=2` ile downsample yapar
- Skip path’te de **projection** ile boyutu eşler

Bu yüzden “basic residual block” öğrenirken şu ayrımı iyi bil:

- **Same-shape block**: identity shortcut
- **Downsample block**: projection shortcut

---

## 6) “Toplama” neden bu kadar işe yarıyor? (sezgi)

Toplama, gradient’in geri akmasını kolaylaştırır. Kabaca:

\begin{aligned}
\frac{\partial y}{\partial x} = \frac{\partial F(x)}{\partial x} + I
\end{aligned}

Buradaki `+ I` (identity) kısmı şu demek:
- `F(x)` tarafı kötü davranıp türevleri küçültse bile, en azından identity yolundan gradient geçer.
- Bu, çok derin ağlarda “öğrenememe/tıkanma” riskini ciddi azaltır.

---

## 7) Basic residual block’ta sık yapılan hatalar

1) **Boyut eşleşmesini unutmak**  
   - Main path `stride=2` yapıp skip path’i identity bırakmak → toplama patlar.

2) **ReLU yerleşimini karıştırmak**  
   - Basic block’ta tipik sıra: (Conv→BN→ReLU→Conv→BN→Add→ReLU)

3) **BN kullanımını kapatıp aynı davranışı beklemek**  
   - BN yoksa daha dikkatli init / LR gerekir.

4) **inplace ReLU yüzünden gereksiz bug’lar**  
   - Çoğu durumda sorun çıkmaz, ama bazı debug senaryolarında işleri zorlaştırabilir.

---

## 8) PyTorch ile implementasyon (temiz ve okunabilir)

Aşağıdaki implementasyon:
- Identity shortcut’u otomatik kullanır
- Boyut gerektiğinde projection’ı otomatik kurar
- ResNet-18/34 tarzına uygundur


In [ ]:
import torch
import torch.nn as nn

class BasicResidualBlock(nn.Module):
    expansion = 1  

    def __init__(self, in_channels: int, out_channels: int, stride: int = 1):
        super().__init__()

        self.conv1 = nn.Conv2d(
            in_channels, out_channels,
            kernel_size=3, stride=stride, padding=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(
            out_channels, out_channels,
            kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.relu = nn.ReLU(inplace=False)

        self.downsample = None
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(identity)

        out = out + identity
        out = self.relu(out)
        return out


### Hızlı şekil testi

İki senaryoya bakıyoruz:

- **Case A (identity shortcut)**: `in=64, out=64, stride=1`
- **Case B (projection shortcut)**: `in=64, out=128, stride=2` (hem kanal hem çözünürlük değişiyor)

Amaç: `out + identity` toplamasının sorunsuz çalıştığını görmek.


In [ ]:
import torch

x = torch.randn(2, 64, 56, 56)

block_identity = BasicResidualBlock(64, 64, stride=1)
y1 = block_identity(x)
print("Identity block:", x.shape, "->", y1.shape)

block_proj = BasicResidualBlock(64, 128, stride=2)
y2 = block_proj(x)
print("Projection block:", x.shape, "->", y2.shape)


## 9) Basic block ne zaman tercih edilir?

- ResNet-18 / ResNet-34 gibi daha “küçük/orta” modellerde kullanılır.
- Daha derin ResNet’lerde (50/101/152) genelde **bottleneck** tercih edilir.
- Basic block’un artısı: **basit, stabil, anlaşılır**.

---


## 10) Kısa özet (tek paragraf)

Basic residual block, iki adet 3×3 conv ile bir dönüşüm `F(x)` üretir ve bunu girdi `x` ile toplayarak `y = F(x) + x` çıkışı oluşturur.  
Boyutlar aynıysa shortcut identity olur; farklıysa 1×1 projection ile eşlenir. Bu toplama, derin ağlarda gradient’in daha rahat akmasını sağlayarak eğitimde tıkanmayı azaltır ve çok derin yapılarda optimizasyonu kolaylaştırır.
